# 循环神经网络的从零开始实现

在本节中，我们将从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上训练一个模型来进行歌词创作。首先，我们读取周杰伦专辑歌词数据集：

In [0]:
import d2ltorch as d2lt
import math
import torch
from torch import nn
import time

(corpus_indices, char_to_idx, idx_to_char, 
 vocab_size) = d2lt.load_data_jay_lyrics()

## one-hot向量

为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。假设词典中不同字符的数量为$N$（即词典大小`vocab_size`），每个字符已经同一个从0到$N-1$的连续整数值索引一一对应。如果一个字符的索引是整数$i$, 那么我们创建一个全0的长为$N$的向量，并将其位置为$i$的元素设成1。该向量就是对原字符的one-hot向量。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [25]:
def one_hot(idx, size, device='cpu'): # 本函数已保存在d2ltorch包中方便以后使用
    batch_size = idx.size(0)
    index = idx.reshape(-1, 1)
    return torch.zeros(batch_size, size).to(device).scatter_(dim=1, index=index, value=1)

one_hot(torch.LongTensor([0, 2]), vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

我们每次采样的小批量的形状是(批量大小, 时间步数)。下面的函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵，矩阵个数等于时间步数。也就是说，时间步$t$的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中$n$为批量大小，$d$为输入个数，即one-hot向量长度（词典大小）。

In [24]:
def to_onehot(X, size, device='cpu'): # 本函数已保存在d2ltorch包中方便以后使用
    return [one_hot(x, size, device) for x in X.long().t()]

X = torch.arange(10).reshape(2, 5)
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, torch.Size([2, 1027]))

## 初始化模型参数

接下来，我们初始化模型参数。隐藏单元个数 `num_hiddens`是一个超参数。

In [20]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
device = d2lt.try_gpu()
print('will use', device)

def get_params():
    def _one(shape):
        return torch.normal(mean=torch.zeros(shape), std=0.01).to(device)
    
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens)
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs)
    # 附上梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    
    for param in params:
        param.requires_grad = True
    return params

will use cuda:0


## 定义模型

我们根据循环神经网络的计算表达式实现该模型。首先定义`init_rnn_state`函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的`Tensor`组成的元组。使用元组是为了更便于处理隐藏状态含有多个`Tensor`的情况。

In [0]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros(batch_size, num_hiddens).to(device), )

下面的`rnn`函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了tanh函数。[“多层感知机”](../chapter_deep-learning-basics/mlp.ipynb)一节中介绍过，当元素在实数域上均匀分布时，tanh函数值的均值为0。

In [0]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(X.mm(W_xh) + H.mm(W_hh) + b_h)
        Y = H.mm(W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出的形状和隐藏状态的形状。

In [22]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size, device)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, torch.Size([2, 1027]), torch.Size([2, 256]))

## 定义预测函数

以下函数基于前缀`prefix`（含有数个字符的字符串）来预测接下来的`num_chars`个字符。这个函数稍显复杂，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [0]:
# 本函数已保存在d2ltorch包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    with torch.no_grad():
        for t in range(num_chars + len(prefix) - 1):
            # 将上一时间片的输出作为当前时间步的输入
            X = to_onehot(torch.Tensor([[output[-1]]]).to(device), vocab_size, device)
            # 计算输出和更新隐藏状态
            (Y, state) = rnn(X, state, params)
            # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
            if t < len(prefix) - 1:
                output.append(char_to_idx[prefix[t + 1]])
            else:
                output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

我们先测试一下`predict_rnn`函数。我们将根据前缀“分开”创作长度为10个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [28]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
           device, idx_to_char, char_to_idx)

'分开祷主椅装招情望点些旧'

## 裁剪梯度

循环神经网络中较容易出现梯度衰减或梯度爆炸。我们会在[“通过时间反向传播”](bptt.ipynb)一节中解释原因。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度

$$ \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}$$

的$L_2$范数不超过$\theta$。

In [0]:
# 本函数已保存在d2ltorch包中方便以后使用
def grad_clipping(params, theta, device):
    norm = torch.Tensor([0]).to(device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data.mul_(theta / norm)

## 困惑度

我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下[“softmax回归”](../chapter_deep-learning-basics/softmax-regression.ipynb)一节中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`。

## 定义模型训练函数

跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

1. 使用困惑度评价模型。
2. 在迭代模型参数前裁剪梯度。
3. 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关讨论可参考[“语言模型数据集（周杰伦专辑歌词）”](lang-model-dataset.ipynb)一节。

另外，考虑到后面将介绍的其他循环神经网络，为了更通用，这里的函数实现更长一些。

In [0]:
# 本函数已保存在d2ltorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2lt.data_iter_random
    else:
        data_iter_fn = d2lt.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter: # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
 
            inputs = to_onehot(X, vocab_size, device)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch_size * num_steps 的向量，这样跟输出的行一一对应
            y = Y.t().flatten().long()
            
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2lt.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.data.item() * torch.numel(y.data)
            n += torch.numel(y.data)
            
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## 训练模型并创作歌词

现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀“分开”和“不分开”分别创作长度为50个字符（不考虑前缀长度）的一段歌词。我们每过50个迭代周期便根据当前训练的模型创作一段歌词。

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

下面采用随机采样训练模型并创作歌词。

In [33]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 65.618297, time 0.12 sec
 - 分开 我不要再爱 我不能再想 我不能再不 我不能再不 我不能再不 我不能再不 我不能再不 我不能再不 我
 - 不分开 一颗 我有你的爱 有不 我不能再不 我不能再不 我不能再不 我不能再不 我不能再不 我不能再不 我
epoch 100, perplexity 9.515193, time 0.12 sec
 - 分开 一颗两步截棍的话斑  印地你的可我有多 从子就这 快对的脚 从反方向 你一定空 我不定 我怎动 一
 - 不分开步 我不能再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
epoch 150, perplexity 2.786049, time 0.13 sec
 - 分开 快使毛 分子的话娘 老有揍重飞离 我的时忆主小的课斑鸠 用力安老斑鸠 腿短毛不多 除天都没有喝水也
 - 不分开吗 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 200, perplexity 1.547270, time 0.12 sec
 - 分开 他金葛一切里 我哼念起国小 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋
 - 不分开期把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 在谁了一切秋动 牵着你的手 一阵
epoch 250, perplexity 1.287400, time 0.12 sec
 - 分开 沙短毛老斑鸠 哼哼哈有 如果我有轻功 飞檐走壁 为人耿直不屈 一身正气 快使用双截棍 哼哼哈兮 快
 - 不分开吗 我不能再想 我不能再想 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 


接下来采用相邻采样训练模型并创作歌词。

In [37]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 57.511170, time 0.11 sec
 - 分开 我想要这 不果了双 在谁用人 在谁用人 在谁用人 在谁用人 在谁用人 在谁用人 在谁用人 在谁用人
 - 不分开 我想要的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏
epoch 100, perplexity 6.501903, time 0.12 sec
 - 分开 我想要这生活 唱着歌 一直她 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的
 - 不分开觉 你已经离开我 不知不觉 我跟了好节奏 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了好节
epoch 150, perplexity 1.952658, time 0.12 sec
 - 分开 这候我 谁怪我 三地安酒 周头一 告诉开没停留 一直在停留 谁让它 告果我的见快就像 隔壁是国术我
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 塞北的客栈人多 牧草有没有 我马儿
epoch 200, perplexity 1.274819, time 0.11 sec
 - 分开 这候我 谁是我 三沉千酒 在杰忆 的路段 有一些风霜 老唱盘 旧皮箱 装满了明信片的铁盒里藏著一片
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250, perplexity 1.149655, time 0.11 sec
 - 分开 问候再 一诉默著阳光 不管到哪里都是晴天 蝴蝶自在飞 花也布满天 一朵一朵因你而香 试图让夕阳飞翔
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 后过了一个秋 后知后觉 我该好好生活 我该好好生


## 小结

* 可以用基于字符级循环神经网络的语言模型来生成文本序列，例如创作歌词。
* 当训练循环神经网络时，为了应对梯度爆炸，可以裁剪梯度。
* 困惑度是对交叉熵损失函数做指数运算后得到的值。


## 练习

* 调调超参数，观察并分析对运行时间、困惑度以及创作歌词的结果造成的影响。
* 不裁剪梯度，运行本节中的代码，结果会怎样？
* 将`pred_period`变量设为1，观察未充分训练的模型（困惑度高）是如何创作歌词的。你获得了什么启发？
* 将相邻采样改为不从计算图分离隐藏状态，运行时间有没有变化？
* 将本节中使用的激活函数替换成ReLU，重复本节的实验。



## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/989)

![](../img/qr_rnn-scratch.svg)